<a href="https://colab.research.google.com/github/moseskim/bert_nlp/blob/main/section_2/04_simple_bert_ko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 간단한 BERT 구현

훈련 완료 모델을 사용해 한국어 문장의 일부를 예측하고, 2개의 문장이 연속되어 있는지 판정합니다.

## 라이브러리 설치

라이브러리인 Transformers를 설치합니다.

In [ ]:
!pip install transformers==4.26.0

## 문장 일부 예측

문자에서 일부 단어를 MASK하고, BERT 모델을 사용해 그 단어를 예측합니다.

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

text = "나는 내일 야구를 관람할 예정입니다"

words = tokenizer.tokenize(text)
print(words)

문장 일부를 MASK합니다.

In [ ]:
msk_idx = 3
words[msk_idx] = "[MASK]"  # 단어를 [MASK]로 치환한다
print(words)

단어를 대응하는 인덱스로 변환합니다.

In [ ]:
import torch

word_ids = tokenizer.convert_tokens_to_ids(words)  # 단어를 인덱스로 변환
word_tensor = torch.tensor([word_ids])  # 텐서로 변환
print(word_tensor)

BERT 모델을 사용해 예측을 수행합니다.

In [ ]:
from google.colab import output
from transformers import BertForMaskedLM

msk_model = BertForMaskedLM.from_pretrained("klue/bert-base")

msk_model.eval()  # 평가 모드
output.clear()

x = word_tensor  # 입력
y = msk_model(x)  # 예측
result = y[0]
print(result.size())  # 결과의 형태

In [ ]:
_, max_ids = torch.topk(result[0][msk_idx], k=5)  # 가장 큰 5개의 값
result_words = tokenizer.convert_ids_to_tokens(max_ids.tolist())  # 인덱스를 단어로 변환
print(result_words)

## 문장이 연속되어 있는지 판정

BERT 모델을 사용해 2개의 문장이 연속되어 있는지 판정합니다.  
다음의 함수 `show_continuity`에서는 2개 문장의 연속성을 판정하고 표시합니다.  

In [7]:
from transformers import BertForNextSentencePrediction

nsp_model = BertForNextSentencePrediction.from_pretrained('klue/bert-base')
nsp_model.eval()  # 평가 모드
output.clear()  # 출력을 표시하지 않는다

In [8]:
def show_continuity(text1, text2):
    tokenized = tokenizer(text1, text2, return_tensors="pt")
    print("Tokenized: ", tokenized)

    # 예측과 결과 표시
    y = nsp_model(**tokenized)  # 예측
    result = torch.softmax(y[0], dim=1)
    print("Result: ", y)

    pred = torch.softmax(y.logits, dim=1)  # Softmax 함수로 확률로 변환
    print(str(result[0][0].item()*100) + "%의 확률로 연속됩니다.")

`show_continuity` 함수에 자연스럽게 이어지는 2개의 문장을 전달합니다.

In [ ]:
text1 = "야구는 무엇입니까?"
text2 = "배트로 공을 치는 스포츠입니다."
show_continuity(text1, text2)

`show_continuity` 함수에 자연스럽게 이어지지 않는 2개의 문장을 전달합니다.

In [ ]:
text1 = "야구는 무엇입니까?"
text2 = "팬 케이크에는 밀가루와 계란과 우유를 사용합니다."
show_continuity(text1, text2)